In [1]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

TODO:

when running with cloud. 
if you dont need to keep the execution logs, change the cloud settings to keep it for a short while
Check that the cache if you need to set the cache to false (especially in plot process)

In [2]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [3]:
# # in the command line. same env
# export GOOGLE_APPLICATION_CREDENTIALS="/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/hai-gcp-genomic-beb9207a40b8.json.json"
# gcloud auth activate-service-account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS

In [4]:
!gcloud auth list
# gcloud config set account


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [5]:
import os
# os.environ["TOWER_ACCESS_TOKEN"] = 'XXX'
os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["TOWER_WORKSPACE_ID"]='247507587970818'


In [6]:
!echo $TOWER_ACCESS_TOKEN
!echo $TOWER_WORKSPACE_ID

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz
247507587970818


In [7]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [10]:
# !chmod +x ./bin/scrna_qc_metrics.py
import os
bin_fld = os.path.join(os.getcwd(),"bin")
bin_fld

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin'

In [11]:
!chmod +x {os.path.join(bin_fld, "scrna_plot_atac_qc_metrics.R")}
!chmod +x {os.path.join(bin_fld, "run_tabix.sh")}
!chmod +x {os.path.join(bin_fld, "plot_insert_size_hist.py")}
!chmod +x {os.path.join(bin_fld, "generate_barcode_metadata.sh")}
!chmod +x {os.path.join(bin_fld, "create_sample_filtered_fragment_file.sh")}
!chmod +x {os.path.join(bin_fld, "compute_tss_script.py")}
!chmod +x {os.path.join(bin_fld, "barcode_rank_functions.R")}
!chmod +x {os.path.join(bin_fld, "atac_barcode_and_subpool.sh")}


In [13]:
# TODO: For a new user: run these commands with your service account for the execution. I am using eila-batch@...

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \

In [14]:
# !nextflow -self-update

In [15]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [12]:
nf_data_base_path = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/'

# Write the configuration file
with open("nextflow_vm.config", "w") as config_file:
    config_file.write(f"""
manifest {{
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_atac.nf'
}}

profiles {{
  conda_profile {{
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
  }}

  docker_local {{
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    executor = 'local'
  }}
  google_batch_profile {{
    workDir = 'gs://eila-nextflow-bucket/work/ATAC'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    executor = 'google-batch'
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = true
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }}
}}

// Define ENV_SYNAPSE_TOKEN as a parameter

params.FASTQS_SPEC_CH = "{nf_data_base_path}nf_wf_fg_singlecell/atac_fastqs_files_vm.csv"
params.CHROMAP_IDX="{nf_data_base_path}nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index"

params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
params.ENV_SYNAPSE_TOKEN = "nextflow secrets get SYNAPSE".execute().text.trim()

params.CHROMAP_TRIM_ADAPTERS = false
params.CHROMAP_REMOVE_PCR_DUPLICATES = false
params.CHROMAP_REMOVE_PCR_DUPLICATES_AT_CELL_LEVEL = false
params.CHROMAP_TN5_SHIFT = false
params.CHROMAP_LOW_MEM = false
params.CHROMAP_BED = false
params.CHROMAP_MAX_INSERT_SIZE = 20
params.CHROMAP_BC_ERROR_THRESHOLD = 20
params.CHROMAP_BC_PROBABILITY_THRESHOLD = 0.20
params.READ_FORMAT = 'r1:0:-1,bc:0:-1'
params.DROP_REPETITIVE_THRESHOLD = false
params.MAPQ_THRESHOLD = false
params.BARCODE_TRANSLATE = 'path'
//barcode_conversion_dict_file
params.BARCODE_CONVERSION_DICT_FILE = '{nf_data_base_path}nf_wf_fg_singlecell/na.na'
params.SUBPOOL='  '
params.ATAC_BARCODE_AND_POOL='atac_barcode_and_subpool.sh'
params.ATAC_FRAGMENTS_CUTOFF = 4
params.RUN_TABIX_SCRIPT='run_tabix.sh'
params.ATAC_TSS_BASES_FLANK = 2000
params.ATAC_TSS_COL_WITH_STRANDS_INFO = 4
params.ATAC_TSS_SMOOTHING_WINDOW_SIZE = 20
params.ATAC_TSS_REGION_BED_FILE='{nf_data_base_path}nf_wf_fg_singlecell/tss_regions.bed'
params.ATAC_TSS_CALCULATION_SCRIPT='compute_tss_script.py'


params.SC_ATAC_QC_PLOT_SCRIPT='scrna_plot_atac_qc_metrics.R'
params.SC_ATAC_QC_PLOT_HELPER_SCRIPT='barcode_rank_functions.R'
params.SC_ATAC_QC_BARCODE_METADATA_FILE='{nf_data_base_path}nf_wf_fg_singlecell/barcode_metadata.txt'
params.SC_ATAC_QC_FRAGMENT_CUTOFF=4
params.SC_ATAC_QC_BARCODE_OUTPUT_FILE='sc_atac_qc_barcode_output_file.png'
    
params.SC_ATAC_BARCODE_METADATA_SCRIPT='generate_barcode_metadata.sh'
params.SC_ATAC_BARCODE_METADATA_FILTERED_BARCODE_STATS='{nf_data_base_path}nf_wf_fg_singlecell/filtered_barcode_stats.csv'
params.SC_ATAC_BARCODE_METADATA_TSS_ENRICHMENT_BARCODE_STATS='{nf_data_base_path}nf_wf_fg_singlecell/tss_enrichment_barcode_stats.csv'

params.SC_ATAC_GENERATE_BARCODE_RANK_PLOT_SCRIPT='plot_insert_size_hist.py'
params.SC_ATAC_GENERATE_BARCODE_RANK_PLOT_PKR='PKR123'

params.SC_JOINT_CELL_BARCODE_PLOT_SCRTIP_PY='joint_cell_plotting.py'
params.SC_JOINT_CELL_BARCODE_PLOT_SCRTIP_R='joint_cell_plotting_density.R'
""")


In [14]:
# !rm -r /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work
# !gsutil rm -r gs://eila-nextflow-bucket/work/ATAC

In [15]:
# !rm .nextflow*

In [17]:
!nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume

# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile


# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile docker_profile


N E X T F L O W  ~  version 23.10.0
Launching `nf_wf_fg_singlecell_atac.nf` [jolly_newton] DSL2 - revision: c741f56046
Dec 18, 2023 2:33:46 AM com.google.auth.oauth2.DefaultCredentialsProvider warnAboutProblematicCredentials
WARN: `params.tss_bases_flank` is defined multiple times -- Assignments following the first are ignored
/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_vm.csv
null
[-        ] process > run_seqspec_modify_atac -
[-        ] process > run_seqspec_print       -
[-        ] process > run_chromap_test        -
[-        ] process > run_chromap_map_to_idx  -
[-        ] process > run_bgzip               -
[-        ] process > run_tabix               -
[-        ] process > run_merge_logs          -
[-        ] process > run_filter_fragments    -
after run_seqspec_modify_atac
after run_seqspec_print
after run_chromap_test
after genome_fasta_ch
after genome_gtf_ch
after genome_chromap_idx download
chromap parameters

In [18]:
!cat .nextflow.log

Dec-18 02:33:41.846 [main] DEBUG nextflow.cli.Launcher - $> nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume
Dec-18 02:33:42.285 [main] INFO  nextflow.cli.CmdRun - N E X T F L O W  ~  version 23.10.0
Dec-18 02:33:42.325 [main] DEBUG nextflow.plugin.PluginsFacade - Setting up plugin manager > mode=prod; embedded=false; plugins-dir=/home/eila/.nextflow/plugins; core-plugins: nf-amazon@2.1.4,nf-azure@1.3.2,nf-cloudcache@0.3.0,nf-codecommit@0.1.5,nf-console@1.0.6,nf-ga4gh@1.1.0,nf-google@1.8.3,nf-tower@1.6.3,nf-wave@1.0.0
Dec-18 02:33:42.344 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Enabled plugins: []
Dec-18 02:33:42.346 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Disabled plugins: []
Dec-18 02:33:42.353 [main] INFO  org.pf4j.DefaultPluginManager - PF4J version 3.4.1 in 'deployment' mode
Dec-18 02:33:42.373 [main] INFO  org.pf4j.AbstractPluginManager - No plugins
Dec-18 02:33:42.407 [main] DEBUG nextflow.config.